In [1]:
#from pwn import *
#p = process(["sage", "-python", "chall.py"])
p = 0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f
q = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
K = GF(p)
a = K(0x0000000000000000000000000000000000000000000000000000000000000000)
b = K(0x0000000000000000000000000000000000000000000000000000000000000007)
E = EllipticCurve(K, (a, b))
G = E(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798, 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)

In [2]:
from pwn import *

#p = process(["sage", "-python", "chall.py"])
p = remote("chal.competitivecyber.club", 6003)
p.recvuntil("Public Key: ")
public_key = p.recvline().strip().decode().split(" : ")
x = int(public_key[0][1:])
y = int(public_key[1])
P = E(x, y)

[x] Opening connection to chal.competitivecyber.club on port 6003
[x] Opening connection to chal.competitivecyber.club on port 6003: Trying 184.72.110.40
[+] Opening connection to chal.competitivecyber.club on port 6003: Done


/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_39991/3219227353.py:5: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.recvuntil("Public Key: ")


In [3]:
import hashlib
def compute_hash(target):
    return int(hashlib.sha256(str(target).encode()).hexdigest(), 16) % q

In [4]:
p.sendline(b"ff")
for _ in range(8):
    p.sendline(b"\x7f\x7f\x7f".hex())


/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_39991/509989189.py:3: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.sendline(b"\x7f\x7f\x7f".hex())


In [5]:
p.recv()

b'Enter separator as a hex value (2 digits): '

In [6]:
separator = b'\xff'
words = ["\x7f\x7f\x7f"]*8
message = separator.join([w.encode() for w in words]) + separator
message_binary = int.from_bytes(message, byteorder='big')
h = compute_hash(message_binary)
R = -h*P
s = 0
while True:
    Ri = int(R.xy()[0] + R.xy()[1])
    if message_binary == (Ri | message_binary):
        break
    R += G
    s += 1
    if s % 100000 == 0:
        print(s)
        print(bin(Ri)[2:].zfill(256))
        print(bin(message_binary)[2:].zfill(256))
p.sendline(hex(R.xy()[0]))
p.sendline(hex(R.xy()[1]))
p.recv()
p.sendline(hex(s))
print(p.recv().strip().split(b'\n')[-1].decode())

100000
1101101100111011011001111101101100011111010111000111010000110111010011111101100000110110010010001011000100000111110111101100111000110111111110011101000001101001001100100100111010011001101000011111111111100000110010110010100001111010000111010111010000110010
0111111101111111011111111111111101111111011111110111111111111111011111110111111101111111111111110111111101111111011111111111111101111111011111110111111111111111011111110111111101111111111111110111111101111111011111111111111101111111011111110111111111111111
200000
1000000111011011001100101010110110100010111111001100101000000110100000010000001010101001001100110000001110101000011001000110001101100001011000000010000001011010110101000010101100111011110100110101011111110001100010110001000010110000100010111111010001010100
01111111011111110111111111111111011111110111111101111111111111110111111101111111011111111111111101111111011111110111111111111111011111110111111101111111111111110111111101111111011111111111111101111111011111110111111

/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_39991/941583079.py:18: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.sendline(hex(R.xy()[Integer(0)]))
/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_39991/941583079.py:19: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.sendline(hex(R.xy()[Integer(1)]))
/var/folders/5r/8j0pby456mv_p468967_cfk00000gn/T/ipykernel_39991/941583079.py:21: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  p.sendline(hex(s))


[x] Receiving all data: 321B
[+] Receiving all data: Done (321B)
[*] Closed connection to chal.competitivecyber.club port 6003
Enter signature s (in hex): Congratulations! Your flag is: PCTF{F33l_th3_d1ff3r3nc3_b3tw33n_c0nc@t_||_b1tw1s3_OR_|_1n_Pyth0n}
